# Optimization of a coarse KPP model

In this notebook we demonstrate the optimization of a coarse KPP model.

1. Free convection
2. Wind-driven shear mixing with neutral buoyancy forcing
3. Wind-driven shear mixing with destabilizing buoyancy forcing
4. Wind-driven shear mixing with stabilizing buoyancy forcing

In [ ]:
using Pkg; Pkg.activate("..")

using OceanTurb, ColumnModelOptimizationProject, PyPlot, Printf, Dao, ColumnModelOptimizationProject.KPPOptimization

datadir = joinpath("..", "data", "idealized")
cases = ["free_convection", "shear_neutral", "shear_unstable", "shear_stable"]
dirs = Dict((c, joinpath(datadir, c)) for c in cases)

examples = Dict(
    "free_convection" => "wind_stress_N256_tau0_Q-75_dTdz0.01_k0.0001_dt0.25_days6_timeseries.jld2",
    "shear_neutral"   => "wind_stress_N256_tau0.1_Q0_dTdz0.001_k0.0001_dt0.25_days6_timeseries.jld2",
    "shear_unstable"  => "wind_stress_N256_tau0.1_Q-75_dTdz0.01_k0.0001_dt0.25_days6_timeseries.jld2",
    "shear_stable"    => "wind_stress_N256_tau0.04_Q10_dTdz0.01_k0.0001_dt0.25_days6_timeseries.jld2" )

examplepaths = Dict((c, joinpath(dirs[c], examples[c])) for c in cases)

┌ Info: Recompiling stale cache file /Users/gregorywagner/.julia/compiled/v1.0/ColumnModelOptimizationProject/HEzqG.ji for ColumnModelOptimizationProject [8137535c-6844-11e9-1caa-ffc4fab9c8ab]
└ @ Base loading.jl:1187


# The model and the data

First we instantiate our column data and our column model.

In [ ]:
column_data = ColumnData(examplepaths["shear_neutral"]);
column_model = KPPOptimization.ColumnModel(column_data, 10*minute, N=10)

[ plot(column_data.T[i], label=@sprintf("\$ t = %0.2f \$ days", column_data.t[i]/day)) for i in (2, 10, 18, 26) ]
xlabel("Temperature (Kelvin)")
ylabel(L"z \, \mathrm{(meters)}")
title("Some LES data")
legend()
removespines("top", "right")

# The 'Negative Log Likelihood' function

Next we define a simple function to compute the negative log likelihood, as well as a
`sampler` that uses the Metropolis algorithm with normally distributed random perturbations
for proposal parameter vectors.

In [ ]:
nll = NegativeLogLikelihood(column_model, column_data, compute_temperature_nll, 1)
sampler = MetropolisSampler(NormalPerturbation([0.05*p for p in default_params]));

# Compute the Markov Chain

In [ ]:
default_params = DefaultFreeParameters(ShearNeutralParameters)
first_link = MarkovLink(nll, default_params)

chain = MarkovChain(100, first_link, nll, sampler);